Import Basic Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Reading The Dataset

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

Merging the dataframes

In [ ]:
data = movies.merge(credit, on='title')

In [ ]:
data.head(1)

In [ ]:
# Required columns from this dataset

# movie_id, title, overview, genres, keywords, cast, crew

data = data[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

Checking for empty data

In [ ]:
data.isnull().sum()

Drop the movies with null overview

In [ ]:
data.dropna(inplace=True)

Doing preprocessing on generes column 

In [ ]:
data.iloc[0].genres

ast.literal_eval converts list in form of string to list

In [ ]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
data['genres'] = data['genres'].apply(convert)

In [ ]:
data['genres']

Preprocessing for keywords column

In [ ]:
data['keywords'] = data['keywords'].apply(convert)

In [ ]:
data['keywords']

Preprocessing on cast column

In [ ]:
def convert1(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            L.append(i['name'])
        else :
            break
        counter += 1
    return L

In [ ]:
data['cast'] = data['cast'].apply(convert1)

In [ ]:
data['cast']

Preprocessing on crew column

In [ ]:
# ...existing code...
import ast

def safe_literal(obj):
    if isinstance(obj, list):
        return obj
    try:
        return ast.literal_eval(obj)
    except (ValueError, SyntaxError, TypeError):
        return []

def convert2(obj):
    items = safe_literal(obj)
    return [d.get('name') for d in items if isinstance(d, dict) and d.get('job') == 'Director']

# Re-run safely (won't break if already converted)
data['crew'] = data['crew'].apply(convert2)
# ...existing code...

In [ ]:
data['crew']

In [ ]:
data['overview'] = data['overview'].apply(lambda x: x.split())
data['overview']

In [ ]:
data['cast'] = data['cast'].apply(lambda x : [i.replace(" ","") for i in x])
data['crew'] = data['crew'].apply(lambda x : [i.replace(" ","") for i in x])
data['keywords'] = data['keywords'].apply(lambda x : [i.replace(" ","") for i in x])
data['genres'] = data['genres'].apply(lambda x : [i.replace(" ","") for i in x])

In [ ]:
data.head()

In [ ]:
data['tags'] = data['overview'] + data['genres'] + data['keywords'] + data['cast'] + data['crew']
new_data = data[['movie_id', 'title', 'tags']]
new_data.head()

In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x:" ".join(x))
new_data['tags']

In [ ]:
new_data.head()

In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x: x.lower())
new_data.head()

Merging similar words

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


def stem(text):
    L = []
    for i in text.split():
        ps.stem(i)
        L.append(ps.stem(i))
    return " ".join(L)

In [ ]:
new_data['tags'] = new_data['tags'].apply(stem)

Vectorization


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_data['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    movie_index = new_data[new_data['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6] 

    for i in movies_list:
        print(new_data.iloc[i[0]].title)

In [ ]:
recommend('Avatar')